In [1]:
import folium
import pandas
import collections
import numpy
import geopy.distance
import itertools
from docplex.mp.model import Model

In [2]:
df_sitios_bog = pandas.read_csv("./data/sitios-de-interes-en-bogota.csv", sep=";")
df_sitios_bog = df_sitios_bog[["Nombre", "Geo Point"]]
df_sitios_bog["Lat"] = df_sitios_bog.apply(lambda row: float(row["Geo Point"].split(",")[0]), axis=1)
df_sitios_bog["Lng"] = df_sitios_bog.apply(lambda row: float(row["Geo Point"].split(",")[1]), axis=1)
df_sitios_bog.drop("Geo Point", axis=1, inplace=True)
df_sitios_bog = df_sitios_bog.head(101)
df_sitios_bog = df_sitios_bog.drop_duplicates(subset="Nombre")
df_sitios_bog = df_sitios_bog.set_index("Nombre")
df_sitios_bog

,Lat,Lng
Nombre,,
Escultura Nave Espacial,4.615251,-74.071534
Estatua Tomas Cipriano de Mosquera,4.597386,-74.076421
Estatua Ricardo Palma,4.602078,-74.066768
Monumento al Almirante José Prudencio Padilla,4.625450,-74.074206
Monumento 21 Ángeles,4.732816,-74.075095
...,...,...
Laboratorios Spaison Ltda,4.681562,-74.117520
Luminex Legrand,4.693044,-74.118457
Doña Juana ESP SA,4.499661,-74.143927


In [3]:
lat_mean = df_sitios_bog["Lat"].mean()
lng_mean = df_sitios_bog["Lng"].mean()

bogota_map = folium.Map(location=[lat_mean, lng_mean], zoom_start=12, tiles="cartodbdark_matter")
for idx, row in df_sitios_bog.iterrows():
    if idx == "Bavaria SA.":
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="yellow", fill=True, fill_color="white", fill_opacity=0.6, popup=f"{idx}").add_to(bogota_map)
    else:
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="blue", fill=True, fill_color="white", fill_opacity=0.8, popup=f"{idx}").add_to(bogota_map)

bogota_map

#### **Conjuntos**

$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$


In [4]:
NODOS = df_sitios_bog.index.to_list()
CLIENTES = df_sitios_bog[df_sitios_bog.index != "Bavaria SA."].index.to_list()

#### **Parámetros**

$$ \text{DISTANCIAS}\_{n1, n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$


In [5]:
df_distancias = pandas.DataFrame()
for nodo1 in NODOS:
    for nodo2 in NODOS:
        df_distancias.loc[nodo1, nodo2] = geopy.distance.geodesic(
            (df_sitios_bog.loc[nodo1, "Lat"], df_sitios_bog.loc[nodo1, "Lng"]), (df_sitios_bog.loc[nodo2, "Lat"], df_sitios_bog.loc[nodo2, "Lng"])
        ).km
        df_sitios_bog

In [6]:
model = Model("TSP")

#### **Variables de Decisión**

$$
x_{n1, n2} = \begin{cases}
1 & \text{Si el vehículo viaja de } n1 \in \text{NODOS} \text{ a } n2 \in \text{NODOS} \\
0 & \text{d.l.c.}
\end{cases}
$$

$$ u\_{n} \in \mathbb{R}^{+} \text{: Carga acumulada del vehículo al llegar al nodo } n \in \text{NODOS} $$


In [7]:
x = model.binary_var_matrix(NODOS, NODOS, name='x')

#### **Función Objetivo**

$$ \text{Minimizar } \sum*{n1 \in \text{NODOS}} \sum*{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1, n2} \cdot x_{n1, n2} $$


In [8]:
model.minimize(model.sum(df_distancias.loc[nodo1, nodo2] * x[nodo1, nodo2] for nodo1 in NODOS for nodo2 in NODOS))

#### **Restricciones**

1. **Prohibición de bucles:**
   $$ \forall n \in \text{NODOS}: x\_{n, n} = 0 $$

2. **Asignación única de salidas para cada cliente:**
   $$ \forall n1 \in \text{NODOS}: \sum*{n2 \in \text{NODOS}} x*{n1, n2} = 1 $$

3. **Asignación única de entradas para cada cliente:**
   $$ \forall n2 \in \text{NODOS}: \sum*{n1 \in \text{NODOS}} x*{n1, n2} = 1 $$

4. **Restricciones para evitar subtours:**
   $$ \forall \text{ ST} \subset \text{NODOS} / 2 \leq |\text{ST}| \leq |\text{NODOS}| - 2 : \sum*{n1 \in \text{NODOS}} \sum*{n2 \in \text{NODOS}} x\_{n1, n2} \leq \left | \text{ST} \right | - 1$$


In [9]:
for nodo in NODOS:
    model.add_constraint(x[nodo, nodo] == 0)

for nodo1 in NODOS:
    model.add_constraint(model.sum(x[nodo1, nodo2] for nodo2 in NODOS if nodo1 != nodo2) == 1)

for nodo1 in NODOS:
    model.add_constraint(model.sum(x[nodo2, nodo1] for nodo2 in NODOS if nodo1 != nodo2) == 1)
    

In [10]:
# Resolver el modelo inicialmente
solution = model.solve()

In [11]:
def get_tour(solution, x, n):
    tour = [0]
    while len(tour) < n:
        for j in range(n):
            if solution.get_value(x[tour[-1], j]) > 0.5:
                tour.append(j)
                break
        if tour[-1] == 0:
            break
    return tour

def get_shortest_subtour(selected_arcs):
    unvisited = NODOS[:]
    cycle = NODOS[:]
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in [arc for arc in selected_arcs if arc[0] == current] if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle
    return cycle

In [12]:
while True:
    selected_arcs = [(n1, n2) for n1 in NODOS for n2 in NODOS if solution.get_value(x[n1, n2]) > 0.5]
    shortest_subtour = get_shortest_subtour(selected_arcs)

    print("\n\nSUBCICLO")
    print(shortest_subtour)
    
    if len(shortest_subtour) < len(NODOS):
        model.add_constraint(model.sum(x[i, j] + x[j, i] for i, j in itertools.combinations(shortest_subtour, 2)) <= len(shortest_subtour) - 1)
        solution = model.solve()
    else:
        break





SUBCICLO
['Matadero Frigo Iberia', 'Doña Juana ESP SA']


SUBCICLO
['Chevron Texaco', 'Bavaria SA.']


SUBCICLO
['Chevron Texaco', 'Avianca']


SUBCICLO
['Centro Educativo Distrital La Chucua Norte', 'Centro Educativo Distrital Nueva Colombia']


SUBCICLO
['Colegio Distrital Mariano Ospina Pérez', 'Laboratorios Spaison Ltda']


SUBCICLO
['Colegio Distrital Magdalena Ortega de Nariño', 'Colegio Distrital Rafael Bernal Jiménez']


SUBCICLO
['Instituto Geográfico Agustín Codazzi - IGAC', 'Notaría 14']


SUBCICLO
['Alcaldía Local de Chapinero', 'Colegio Distrital Manuela Beltrán']


SUBCICLO
['Alcaldía Local de Sumapaz', 'Munal']


SUBCICLO
['Iglesia de San Agustín', 'Ministerio de Hacienda']


SUBCICLO
['Iglesia Nuestra Señora de La Paz', 'Sub Estación de Policía de Muzu']


SUBCICLO
['Parroquia Señor de La Buena Esperanza', 'CADE Bosa']


SUBCICLO
['Parroquia Santa Cecilia', 'Estación de Bomberos de Venecia']


SUBCICLO
['Iglesia Santa Juana', 'Luminex Legrand']


SUBCICLO
['Parroquia 

In [14]:
solution.objective_value

156.21552612794568

In [15]:
depot = "Bavaria SA."
for cliente in CLIENTES:
    if x[depot, cliente].solution_value > 0.9:
        print(f"==== Ruta ====")
        ruta = [depot, cliente]
        r = f"{depot} -> {cliente} -> "

        cliente_actual = cliente
        while x[cliente_actual, depot].solution_value < 0.9:
            for siguiente_cliente in CLIENTES:
                if x[cliente_actual, siguiente_cliente].solution_value > 0.9:
                    break
            ruta.append(siguiente_cliente)
            r += str(siguiente_cliente) + " -> "
            cliente_actual = siguiente_cliente

        ruta.append(depot)
        r += depot
        print(r)

==== Ruta ====
Bavaria SA. -> Chevron Texaco -> Avianca -> Ministerio de Defensa Nacional -> Instituto Colombiano de Desarrollo Rural - INCODER -> Centro Religioso Policía Nacional -> Clínica Federman -> Centro Educativo Distrital Alemania -> Iglesia de Jesucristo de los Santos de los Últimos Días -> Hospital Universitario Barrios Unidos Mederi -> Colegio Distrital Rafael Bernal Jiménez -> Colegio Distrital Magdalena Ortega de Nariño -> Templo Adventista Séptimo Día -> Colegio Distrital Mariano Ospina Pérez -> Laboratorios Spaison Ltda -> Luminex Legrand -> Iglesia Santa Juana -> CAMI San Pablo -> Iglesia Nuestra Señora de Fontibón -> Protex SA. -> CAMI Emaus -> Fábrica Indubo -> Estación de Bomberos Garcés Navas -> Iglesia Jesucristo de Los Santos de Los Últimos Días -> Centro Educativo Distrital Nueva Colombia -> Centro Educativo Distrital La Chucua Norte -> Monumento 21 Ángeles -> Plaza Fundacional de Suba -> Estación de Bomberos Suba -> Colegio Distrital Chorrillos -> Club Bellavis

In [16]:
def get_bearing(p1, p2):
    long_diff = numpy.radians(p2.lon - p1.lon)
    lat1 = numpy.radians(p1.lat)
    lat2 = numpy.radians(p2.lat)
    x = numpy.sin(long_diff) * numpy.cos(lat2)
    y = numpy.cos(lat1) * numpy.sin(lat2) - (numpy.sin(lat1) * numpy.cos(lat2) * numpy.cos(long_diff))
    bearing = numpy.degrees(numpy.arctan2(x, y))
    if bearing < 0:
        return bearing + 360
    return bearing


def get_arrows(locations, color="black", size=4, n_arrows=3, weight_arrow=3):
    Point = collections.namedtuple("Point", field_names=["lat", "lon"])
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    rotation = get_bearing(p1, p2) - 90
    arrow_lats = numpy.linspace(p1.lat, p2.lat, n_arrows + 2)[1 : n_arrows + 1]
    arrow_lons = numpy.linspace(p1.lon, p2.lon, n_arrows + 2)[1 : n_arrows + 1]
    arrows = []
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(
            folium.RegularPolygonMarker(
                location=points,
                color=color,
                weight=weight_arrow,
                fill=True,
                fill_color=color,
                fill_opacity=1,
                number_of_sides=3,
                radius=size,
                rotation=rotation,
            )
        )
    return arrows

In [17]:
lat_mean = df_sitios_bog["Lat"].mean()
lng_mean = df_sitios_bog["Lng"].mean()

bogota_map = folium.Map(location=[lat_mean, lng_mean], zoom_start=12, tiles="cartodbdark_matter")
for idx, row in df_sitios_bog.iterrows():
    if idx == "Bavaria SA.":
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=10, color="yellow", fill=True, fill_color="white", fill_opacity=0.6, popup=f"{idx}").add_to(bogota_map)
    else:
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="blue", fill=True, fill_color="white", fill_opacity=0.8, popup=f"{idx}").add_to(bogota_map)

for i in range(len(ruta) - 1):
    coordinates = [
        df_sitios_bog.loc[ruta[i], ["Lat", "Lng"]].values.tolist(),
        df_sitios_bog.loc[ruta[i + 1], ["Lat", "Lng"]].values.tolist(),
    ]

    color = "gray"
    weight = 2

    pl = folium.PolyLine(coordinates, color=color, weight=weight)
    bogota_map.add_child(pl)

    arrows = get_arrows(locations=coordinates, color=color, size=2, n_arrows=1)
    for arrow in arrows:
        arrow.add_to(bogota_map)

bogota_map